In [1]:
import pandas as pd
from tree_sitter import Language, Parser
from language_data import LANGUAGE_METADATA
from process import DataProcessor

In [2]:
Language.build_library('build/ts-python.dll',['tree-sitter-python'])

False

In [3]:
language = 'python'
PY_LANGUAGE = Language('build/ts-python.dll', 'python')

DataProcessor.PARSER.set_language(PY_LANGUAGE)
processor = DataProcessor(language=language,
                          language_parser=LANGUAGE_METADATA[language]['language_parser'])

In [4]:
definitions = {}
definitions['numpy'] = processor.process_definitions('numpy/numpy', 'D:\\work4\\numpy\\numpy\\', ext=LANGUAGE_METADATA[language]['ext'])
definitions['sklearn'] = processor.process_definitions('scikit-learn/scikit-learn', 'D:\\work4\\scikit-learn\\scikit-learn\\', ext=LANGUAGE_METADATA[language]['ext'])

In [5]:
calls = []
calls += processor.process_calls('scikit-learn/scikit-learn', 'D:\\work4\\scikit-learn\\scikit-learn\\', ext=LANGUAGE_METADATA[language]['ext'], library_candidates=definitions)
calls += processor.process_calls('numpy/numpy', 'D:\\work4\\numpy\\numpy\\', ext=LANGUAGE_METADATA[language]['ext'], library_candidates=definitions)

In [6]:
function_definitions = {}
for lib in definitions:
    function_definitions[lib] = pd.DataFrame(definitions[lib])

function_calls = pd.DataFrame(calls)

In [7]:
for called_lib in function_definitions:
    for called_function in function_definitions[called_lib].itertuples():
        for call in function_calls.loc[(function_calls['call_nwo'] == called_function.nwo) & 
                                       (function_calls['call_path'] == called_function.path) & 
                                       (function_calls['called_identifier'] == called_function.identifier)].itertuples():
            for calling_lib in function_definitions:
                for calling_function in (function_definitions[calling_lib].loc[(function_definitions[calling_lib]['nwo'] == call.calling_nwo) & 
                                                                               (function_definitions[calling_lib]['path'] == call.calling_path) & 
                                                                               (function_definitions[calling_lib]['start_point'] <= call.calling_start_point) & 
                                                                               (function_definitions[calling_lib]['end_point'] >= call.calling_end_point)]).itertuples():
                    function_definitions[called_lib].at[called_function.Index, 'called'].append({'libname':calling_lib, 'identifier':calling_function.identifier, 'path':calling_function.path, 'start_point':call.calling_start_point,'end_point':call.calling_end_point})
                    function_definitions[calling_lib].at[calling_function.Index, 'calls'].append({'libname':called_lib, 'identifier':called_function.identifier, 'path':called_function.path, 'start_point':call.call_start_point,'end_point':call.call_end_point})

In [18]:
function_definitions['sklearn']

,nwo,sha,path,identifier,start_point,end_point,called,calls
0,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\asv_benchmarks\benchmarks\common.py,get_from_config,12,51,[],[]
1,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\asv_benchmarks\benchmarks\common.py,get_estimator_path,54,62,[],[]
2,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\asv_benchmarks\benchmarks\common.py,clear_tmp,65,69,[],[]
3,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\asv_benchmarks\benchmarks\datasets.py,_blobs_dataset,18,25,[],"[{'libname': 'sklearn', 'identifier': 'make_bl..."
4,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\asv_benchmarks\benchmarks\datasets.py,_20newsgroups_highdim_dataset,29,37,[],"[{'libname': 'sklearn', 'identifier': 'fetch_2..."
...,...,...,...,...,...,...,...,...
1199,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\sklearn\_build_utils\pre_build_helpers.py,basic_check_build,93,105,"[{'libname': 'sklearn', 'identifier': 'cythoni...",[]
1200,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\sklearn\_build_utils\__init__.py,_check_cython_version,21,34,"[{'libname': 'sklearn', 'identifier': 'cythoni...",[]
1201,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\sklearn\_build_utils\__init__.py,cythonize_extensions,37,74,[],"[{'libname': 'sklearn', 'identifier': 'check_o..."
1202,scikit-learn/scikit-learn,0aee596bb32136df8c68371d696770251c7d14a0,\sklearn\_build_utils\__init__.py,gen_from_templates,77,95,[],[]
